### Can we show a correlation between major and region based on data about early, mid, and late career graduates?
### Do Cal Poly Graduates follow national distribution trends based on major?
### How would a computer decide/what would a computer prioritize when deciding where to move.
### What feature is most important on deciding where to move after graduation?



In [10]:
# Package imports
import pandas as pd
import numpy as np

In [11]:
# Data Preprocessing

df_major_salary = pd.read_csv('cp_major_sal.csv')

df_degrees_over_career = pd.read_csv('degrees-that-pay-back.csv')

lis = 'Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont, New Jersey, New York, Pennsylvania, Illinois, Indiana, Michigan, Ohio, Wisconsin, Iowa, Kansas, Minnesota, Missouri,  Nebraska, North Dakota, South Dakota, Delaware, District of Columbia, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia, West Virginia, Alabama, Kentucky, Mississippi, Tennessee, Arkansas, Louisiana, Oklahoma, Texas, Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah, Wyoming, Alaska, California, Hawaii, Oregon, Washington'
my_states = lis.split(', ')
my_states =  [*set(my_states)]
my_states


df_living_wage = pd.read_csv('livingwage.csv')
df_living_wage = df_living_wage[['city','state','one_adult_no_kids_living_wage', 'land_area_sqmi']]
# On average, a full-time employee in the United Stats works 1,801 hours per year, or 37.5 hours per week, 
# which is more than other OECD countries. Source: https://clockify.me/working-hours#:~:text=On%20
# average%2C%20a%20full%2Dtime%20employee%20in%20the%20United%20Stats,more%20than%20other%20OECD%20countries.
df_living_wage['one_adult_no_kids_living_salary'] = df_living_wage['one_adult_no_kids_living_wage']*1801


In [12]:
df_homes = pd.read_csv('united_states_2022.csv')
df_homes = df_homes[['price','addressState','beds','baths','area', 'zestimate', 'homeType']]
df_homes = df_homes.dropna()
df_homes['addressState'].value_counts()

states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}


morts = {
    'AK': 2060,
    'AL': 1171,
    'AR': 1335,
    'AZ': 1516,
    'CA': 2060,
    'CO': 1516,
    'CT': 1735,
    'DC': 1396,
    'DE': 1396,
    'FL': 1396,
    'GA': 1396,
    'HI': 2060,
    'IA': 1209,
    'ID': 1516,
    'IL': 1207,
    'IN': 1207,
    'KS': 1209,
    'KY': 1171,
    'LA': 1335,
    'MA': 1735,
    'MD': 1396,
    'ME': 1735,
    'MI': 1207,
    'MN': 1209,
    'MO': 1209,
    'MS': 1171,
    'MT': 1516,
    'NC': 1396,
    'ND': 1209,
    'NE': 1209,
    'NH': 1735,
    'NJ': 1657,
    'NM': 1516,
    'NV': 1516,
    'NY': 1657,
    'OH': 1207,
    'OK': 1335,
    'OR': 2060,
    'PA': 1657,
    'RI': 1735,
    'SC': 1396,
    'SD': 1209,
    'TN': 1171,
    'TX': 1335,
    'UT': 1516,
    'VA': 1396,
    'VT': 1735,
    'WA': 2060,
    'WI': 1207,
    'WV': 1396,
    'WY': 1516
}
df_homes['State'] = df_homes['addressState'].map(states)
df_homes['Average Mortgage'] = df_homes['addressState'].map(morts)

/tmp/ipykernel_381878/151017707.py:1: DtypeWarning: Columns (0,1,10,15,16,17,18,19,21,23,25,26,27,30,31,32,33,34,37,38,40,41,48,51,52,53,54,56,57,58,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_homes = pd.read_csv('united_states_2022.csv')


In [13]:
df = df_living_wage.merge(df_homes,  left_on='state', right_on='State')
df

,city,state,one_adult_no_kids_living_wage,land_area_sqmi,one_adult_no_kids_living_salary,price,addressState,beds,baths,area,zestimate,homeType,State,Average Mortgage
0,New York,New York,20.00,300.5,36020.00,"$675,000",NY,4.0,3.0,2444.0,675000.0,SINGLE_FAMILY,New York,1657.0
1,New York,New York,20.00,300.5,36020.00,"$739,000",NY,3.0,3.0,1583.0,671000.0,SINGLE_FAMILY,New York,1657.0
2,New York,New York,20.00,300.5,36020.00,"$160,000",NY,1.0,1.0,750.0,459800.0,SINGLE_FAMILY,New York,1657.0
3,New York,New York,20.00,300.5,36020.00,"$749,999",NY,4.0,3.0,2200.0,789700.0,MULTI_FAMILY,New York,1657.0
4,New York,New York,20.00,300.5,36020.00,"$439,000",NY,3.0,2.0,1910.0,439000.0,MULTI_FAMILY,New York,1657.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459477,Boise,Idaho,14.12,84.0,25430.12,"$515,900",ID,3.0,2.0,1450.0,478800.0,SINGLE_FAMILY,Idaho,1516.0
459478,Boise,Idaho,14.12,84.0,25430.12,"$450,000",ID,2.0,3.0,1552.0,446800.0,TOWNHOUSE,Idaho,1516.0
459479,Boise,Idaho,14.12,84.0,25430.12,"$2,050,000",ID,4.0,5.0,4760.0,1922500.0,SINGLE_FAMILY,Idaho,1516.0
459480,Boise,Idaho,14.12,84.0,25430.12,"$1,269,000",ID,2.0,2.0,2120.0,1244500.0,SINGLE_FAMILY,Idaho,1516.0


In [14]:
df_degrees_over_career = df_degrees_over_career.set_index('Undergraduate Major')


In [17]:
for col in list(df_degrees_over_career):
    if col != 'Percent change from Starting to Mid-Career Salary':
        df_degrees_over_career[col] = df_degrees_over_career[col].str.strip('$')
        df_degrees_over_career[col] = df_degrees_over_career[col].str.replace(',', '')
    df_degrees_over_career[col] = df_degrees_over_career[col].astype(float)

In [8]:
# 5 Majors Closest to Accounting Wage Wise For Career Estimates
x = df_degrees_over_career.iloc[0]
db = df_degrees_over_career.drop(df_degrees_over_career.index[0]) # Make database of all users except user 1
sims = db.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
sorted_sims = sims.sort_values()[::-1]
N=10
userIds = sorted_sims.dropna().iloc[:N].index
df_degrees_over_career.loc[userIds]

In [ ]:
# i feel like here would be a good spot to talk about if any of these share clustering 

# then maybe we can loop in housing data, maybe we ask about directions on tuesday?

# with these in mind, if we see a cluster or not, are we seeing these majors cluster in areas they can afford?

# TODO: make some empty cells to do some MAE evaluation, link in clustering, add another component?